In [160]:
import pandas as pd
import sweetviz
import re
import random
import pandas as pd
import glob
import warnings
import unidecode
import numpy as np
warnings.filterwarnings('ignore')

In [161]:
def missing_value_of_data(data):
    total=data.isnull().sum().sort_values(ascending=False)
    percentage=round(total/data.shape[0]*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [162]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [163]:
def unique_values_in_column(data,feature):
    unique_val=pd.Series(data.loc[:,feature].unique())
    return pd.concat([unique_val],axis=1,keys=['Unique Values'])

In [164]:

def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [165]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [166]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [167]:
#
def sentence_splitter(content):
    # splitters = ["፡፡","።","?","::","!"]
    sentence = []
    sentences = re.split("([፡፡።?!])",content)
    #print(sentences)
    w = len(sentences)
    for i in np.arange(0,w,2):
        if(len("".join(sentences[i:i+2])) > 1):
            if("".join(sentences[i:i+2])[-1] == "፡"):
                sentence.append("፡".join(sentences[i:i+2]))
            else:
                sentence.append("".join(sentences[i:i+2]))
    return sentence
    
    #return [sentence.strip() for sentence in sentences if len(sentence) > 1]

In [168]:
def remove_English_letters(content):
    for idx,cnt in enumerate(content):
        content[idx] = re.sub(r'[a-zA-Z]','', cnt)
    return content


def normalize(data):
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    return(normal.decode())  

In [169]:
remove = ['\xa087','\n','\xa0', '|', '/','፤',':','፣','«','»','...','..']
May_news_DF = pd.DataFrame()
May_news  = glob.glob("2020_05_amharic/output/*.csv")
for news in May_news:
    data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"])
    
    #print(missing_value_of_data(data))
    
    #print(count_values_in_column(data, 'Content'))
    
    #print(unique_values_in_column(data,'Content'))
    
    #Checking Duplicated sentences
    
    #print(duplicated_values_data(data))
    
    
    #Removing duplicated sentences from Content
    data = data[data.Content.duplicated()==False].reset_index()
    
    #Checking Content after removing duplicated sentences
    #print(duplicated_values_data(data))
    
    for token in remove:
        data.Content = data.Content.apply(lambda x: x.replace(token, ' ')) 
    #Removing the url from Content
    data.Content = data.Content.apply(lambda x: remove_url(x))
    
    
    #Shuffling the sentences under Content
    random.shuffle(data.Content)
    data = data.drop(['index','ID'], axis=1)
    
    #add arbitrary ID
    data.insert(0,"Arbitrary_ID", len(May_news_DF.index)+data.index)
    

    
    #Merging the Dataframes
    May_news_DF = pd.concat([May_news_DF,data])
    

#Before removing duplications
print("Before Removing Duplicated Sentences")
print(duplicated_values_data(May_news_DF))

#Removing duplicated sentences from Content
May_news_DF = May_news_DF[May_news_DF.Content.duplicated()==False].reset_index()
May_news_DF = May_news_DF.drop(['index'], axis=1)
print("="*40,"\nAfter Removing Duplicated Sentences")
print(duplicated_values_data(May_news_DF))


#Splitting the sentences
May_news_DF.Content = May_news_DF.Content.apply(lambda x: sentence_splitter(x))

#Remove Latin-alphabets from content 
May_news_DF.Content = May_news_DF.Content.apply(lambda x: remove_English_letters(x))

Before Removing Duplicated Sentences
        Columns  Duplicate count
0  Arbitrary_ID                0
1           URL           225465
2          Date           305100
3         Media           250168
4       Content           224889
After Removing Duplicated Sentences
        Columns  Duplicate count
0  Arbitrary_ID                0
1           URL            24572
2          Date            80211
3         Media            40278
4       Content                0


In [81]:
May_news_DF.to_csv("MayNews.csv")

In [101]:
May_news_DF.Content.to_csv("MayNews_Content.csv",index=False)

In [103]:
#Number of Sentences
Sentence_Number = []
for x in May_news_DF.Content:
    Sentence_Number.append(len(x))

sum(Sentence_Number)

1820642

In [170]:
May_news_DF.head()

,Arbitrary_ID,URL,Date,Media,Content
0,0,https://www.dw.com/am/%E1%8B%AD%E1%8B%98%E1%89...,2020-05-10,ይዘት | DW,"[የበኒ ሻንጉል የፖለቲካ ፓርቲዎች ዉዝግብ አውዲዮውን ያዳምጡ።, 03 01 ]"
1,1,https://www.bbc.com/amharic,2020-05-10,ዜና - BBC News አማርኛ,"[ዶ ር መረራና 114ቱ እስረኞች ተፈቱ ሌሎቹስ ?, 19.01.2018..."
2,2,https://amharic.voanews.com//z/4011,2020-05-10,ጋቢና ቪኦኤ - የፕሮግራሙ ተከታታይ ክፍሎች,[1440ኛው የኢድ አል አደሀ አከባበር 11.08.2019 1440ኛው የኢድ...
3,3,http://www.aigaforum.com/geez-index.php,2020-05-10,"Aiga Forum, an Ethiopian forum for news and vi...","[የኢሕአዴግ ውሕደት ጸድቋል የሕወሓት እጣ ፈንታስ?, 24.11.2019 ]"
4,4,https://www.shegerfm.com/,2020-05-10,ሸገር | የፊት ገጽ,[ የታፈነዉ የተቃዉሞና እንቅስቃሴዋ የተቋረጠዉ ጎንደር ባሳለፍነዉ ሳምን...


In [171]:
May_news_DF.Content[10]

['   ኢትዮጵያዉያን  ስደት  ስደተኞችና መከራቸዉ 17.07.2017 ሰደተኞች  የሰደተኞቹ ልጆች የየተሰደዱበት ሐገርና ሕዝብም  የእልፍ ሲሉ እልፍ  ይገኛል ዉጤት መሆናቸዉ የሚነገር-የሚመሰከርበት የዚሕ ዘመን ፖለቲከኛ የዘመኑን ስደተኛን አይንሕ ላፈር ማለቱ እንጂ  ሰቀቀኑ።',
 'የሳዑዲ አረቢያ ነገስታት የተለዩ አይደሉም።',
 'እርምጃቸዉ ግን ጠንክሯል።',
 'የኢትዮጵያዉኑ ስደተኞች መከራም ከፍቷል።',
 ' አዉሮጶች እንደሚወገዙ እንደሚወቀሱባቸዉ ብዙ ነገሮች ሁሉ የሚደነቁ የሚመሰገኑባቸዉ አያሌ መርሕ  እርማጃዎች አሏቸዉ።',
 'ለልጅ የሚያደርጉትን አለማወደስ ግን በርግጥ አይቻልም።',
 'አብዛኞቹ የአዉሮጳ መንግሥታት ለልጅ ገንዘብ ይከፍላሉ።',
 'የሳዑዲ አረቢያ ነገሥታት በቅርቡ የነደፉት ዕቅድ እስከ 2030 (እንደ ጎርጎሪያኑ አቆጣጠር ነዉ) ባለዉ ጊዜ ሐገር-ሕዝባቸዉን እንደ አዉሮጳ ለማዘምን ያለመ ነዉ ይላሉ።',
 'እኩልነትን ለደካሞች አዘኔታን ለድሆች እርዳትን የሚሰብከዉ ትልቅ ሐይማኖት የፈለቀባት ሐገር ነገስታት ሐገርን በማዘመን ስም እመጫት እርጉዝ በሽተኛ አዛዉንትን ሳይቀር ስደተኛን በገፍ ያባርራሉ።',
 'ለማባረር ምክንያት ያጡለትን በየልጁ ቁጥር ልክ ገንዘብ ያስከፍላሉ።',
 'የሪያዶች እርምጃ መነሻ  የኢትዮጵያዉን ስደተኞች አበሳ መድረሻችን ነዉ።',
 '                               ሁለቱ  ዛሬ እንጂ ያኔ የአንድ ክፍለ-ግዛት  የአንድ ዘመን ዉልድ  ግን የማይተዋወቁ የቅርብ-ሩቅ ልጆች ነበሩ።',
 'አንደኛዉ የሙዚቃ ፍቅር ጥማቱን ሌላዉ በርትቶ የመስራት ፍላጎቱን ለማርካት  ሁለቱም እኩል ኑሮን የመለወጥ ዓላማን ለማሳካት የመጀመሪያዉ ቀድሞ  ሁለተኛዉ ተከትሎ  ቀዳሚዉ ከየጁ አዲስ አበባ  ተከታዩ ከወልዲያ ጂዳ ገቡ።',
